In [26]:
import tensorflow as tf

#Block of layers with bool for if grad = true
#Grad = not true, no flatten and FCL

class GroupedLayer(tf.keras.Model):
    def __init__(self, filters, kernel_size, final_output):
        super(GroupedLayer, self).__init__()
        self.hidden_1 = tf.keras.layers.Conv2D(filters, kernel_size, strides=(1,1), padding="same", activation='ReLU')
        self.hidden_2 = tf.keras.layers.Conv2D(filters, kernel_size, strides=(1,1), padding="same", activation='ReLU')
        self.hidden_output = tf.keras.layers.Dense(units=final_output, activation='ReLU')
        
    def call(self, inputs):
        inputs = self.hidden_1(inputs)
        inputs = self.hidden_2(inputs)
        return inputs
    
    def call_output(self, inputs):
          return self.hidden_output(tf.keras.layers.Flatten()(self.call(inputs)))

class GroupedLayerInput(tf.keras.Model):
    def __init__(self, filters, kernel_size, final_output):
        super(GroupedLayerInput, self).__init__()
        self.input_block = tf.keras.Sequential(
            [
                tf.keras.Input(shape=(32, 32, 3)),
                tf.keras.layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same", activation="ReLU"),
                tf.keras.layers.MaxPooling2D(pool_size=2),

            ],
            name="input",
        )
        self.hidden = GroupedLayer(64, (3,3), 6)
        
    def call(self, inputs):
        inputs = self.input_block(inputs)
        inputs = self.hidden.call(inputs)
        return inputs
    
    def call_output(self, inputs):
          return self.hidden(tf.keras.layers.Flatten()(self.call(inputs)))

grouped_block = GroupedLayer(64,3,10)

In [94]:
loss_tracker = tf.keras.metrics.Mean(name="loss")
class ForcedLearnerClass(tf.keras.Model):
    
    def __init__(self, output_size):
        super(ForcedLearnerClass, self).__init__()
        self.input_block = tf.keras.Sequential([
            GroupedLayerInput(filters=64,kernel_size=(3,3), final_output=6)
        ])
        
        self.training_block_1 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
        self.training_block_2 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
        self.training_block_3 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
        self.training_block_4 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
    
    def compile(self, optimizer, loss_fn):
        super(ForcedLearnerClass, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        
    def call(self, x):
        x = self.input_block(x)
        x = self.training_block_1(x)
        x = self.training_block_2(x)
        x = self.training_block_3(x)
        return self.training_block_4.call_output(x)
        
    def train_step(self, data):
        pred_output, y = data
        
        with tf.GradientTape(persistent=True) as tape:
            pred_output_back_input = self.input_block(pred_output)
            pred_output = self.input_block.call(pred_output)
            
            for i in range(2):
                pred_output_back_1 = self.training_block_1[i].call_output(pred_output)
                pred_output = self.training_block_1[i].call(pred_output)
                
            for i in range(2):
                pred_output_back_2 = self.training_block_2[i].call_output(pred_output)
                pred_output = self.training_block_2[i].call(pred_output)
                
            for i in range(2):
                pred_output_back_3 = self.training_block_3[i].call_output(pred_output)
                pred_output = self.training_block_3[i].call(pred_output)
                
            for i in range(2):
                pred_output_back_4 = self.training_block_4[i].call_output(pred_output)
                pred_output = self.training_block_4[i].call(pred_output)
            
            loss_input = self.loss_fn(y, pred_output_back_input)
            loss_1 = self.loss_fn(y, pred_output_back_1)
            loss_2 = self.loss_fn(y, pred_output_back_2)
            loss_3 = self.loss_fn(y, pred_output_back_3)            
            loss_4 = self.loss_fn(y, pred_output_back_4)   
    
        #mae_metric = keras.metrics.MeanAbsoluteError(name="mae")
        gradients_input = tape.gradient(loss_input, self.input_block.trainable_variables)
        gradients_1 = tape.gradient(loss_1, self.training_block_1.trainable_variables)
        gradients_2 = tape.gradient(loss_2, self.training_block_2.trainable_variables)
        gradients_3 = tape.gradient(loss_3, self.training_block_3.trainable_variables)
        gradients_4 = tape.gradient(loss_4, self.training_block_4.trainable_variables)

        #Update weights
        self.optimizer.apply_gradients(zip(gradients_input, self.input_block.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients_1, self.training_block_1.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients_2, self.training_block_2.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients_3, self.training_block_3.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients_4, self.training_block_4.trainable_variables))
         
        #Compute our own metrics
        loss_tracker.update_state(loss_4)
        #mae_metric.update_state(y, y_pred)
        #"mae": mae_metric.result()
        return {"loss": loss_tracker.result()}

In [20]:
import tensorflow as tf
training_block_1 = tf.keras.Sequential(
            [
                GroupedLayer(64, (64,64), 6) for _ in range(2)
            ]
        )

training_block_1.build((16,224,224,3))

training_block_1.summary()
training_block_1.trainable_variables
training_block_1.compile()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 grouped_layer_23 (GroupedLa  (16, 224, 224, 64)       17563776  
 yer)                                                            
                                                                 
 grouped_layer_24 (GroupedLa  (16, 224, 224, 64)       33554560  
 yer)                                                            
                                                                 
Total params: 51,118,336
Trainable params: 51,118,336
Non-trainable params: 0
_________________________________________________________________


In [ ]:
network_block = GroupedLayerInput(2, (3,3), 6)

network_block.build((16,224,224,3))

In [95]:
forced_learner = ForcedLearnerClass(output_size=6)
forced_learner.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=tf.keras.losses.Huber(),
)

In [96]:
batch_size = 64
(x_train, Y_train), (x_test, Y_test) = tf.keras.datasets.cifar10.load_data()

In [97]:
import numpy as np
all_digits = np.concatenate([x_train, x_test])
all_output = np.concatenate([Y_train, Y_test])
all_digits = all_digits.astype("float32") / 255.0

In [98]:
all_digits = np.reshape(all_digits, (-1, 32, 32, 3))
dataset_img = tf.data.Dataset.from_tensor_slices(all_digits)
dataset_out = tf.data.Dataset.from_tensor_slices(all_output)
dataset = tf.data.Dataset.zip((dataset_img, dataset_out))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [99]:
history = forced_learner.fit(dataset, epochs=10)

Epoch 1/10


InvalidArgumentError:  Incompatible shapes: [64,8,8,64] vs. [64,1]
	 [[node gradient_tape/huber_loss/BroadcastGradientArgs_2
 (defined at /tmp/ipykernel_14588/4073329151.py:57)
]] [Op:__inference_train_function_23195]

Errors may have originated from an input operation.
Input Source operations connected to node gradient_tape/huber_loss/BroadcastGradientArgs_2:
In[0] gradient_tape/huber_loss/Shape_7:	
In[1] gradient_tape/huber_loss/Shape_8:

Operation defined at: (most recent call last)
>>>   File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/jack/ml/my_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/home/jack/ml/my_env/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
>>>     

In [ ]:
print(dataset.take(1))

In [ ]:
for i in range(2):
    print(i)

In [93]:
for i in range(2):
    print(i)

0
1


In [ ]:
input_block = GroupedLayerInput(filters=64,kernel_size=(3,3), final_output=6)
training_block_1 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
training_block_2 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
training_block_3 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]
training_block_4 = [GroupedLayer(64, (64,64), 6) for _ in range(2)]

In [ ]:
training_block_1[1].build((16,224,224,3))
training_block_1[1].non_trainable_weights

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
epochs = 2

# Iterate over epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
           
        
            #Compute Loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))
